<a href="https://colab.research.google.com/github/sa-shah/QuDPy/blob/master/Holstein_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qudpy

In [ ]:
from qutip import *  # for quantum dynamics
import numpy as np  # for numerical manipulations
from qudpy.Classes import *  # for nonlinear spectroscpy
import qudpy.plot_functions as pf  # for plotting nonlinear spectra
import matplotlib.pyplot as plt
import ufss  # for double-sided diagram generation
print('ok')

In [ ]:
# Generating the 3rd order double-sided diagrams for rephasing and non-rephasing response
# in impulsive regime.

DG = ufss.DiagramGenerator
R3rd = DG()
# setting condition for rephasing diagrams R1,2,3
R3rd.set_phase_discrimination([(0, 1), (1, 0), (1, 0)])
# Set the pulse interval
t_pulse = np.array([-1, 1])
R3rd.efield_times = [t_pulse]*4
# Creating diagrams for pulse arrival times 0, 100, 200 and detection time 300.
[R3, R1, R2] = R3rd.get_diagrams([0, 100, 200, 300])
rephasing = [R1, R2, R3]
print('the rephasing diagrams are R1, R2 and R3 ', rephasing)

# setting conditions for and generating non-rephasing diagrams R4, R5 and R6
R3rd.set_phase_discrimination([(1, 0), (0, 1), (1, 0)])
[R6, R4, R5] = R3rd.get_diagrams([0, 100, 200, 300])
nonrephasing = [R4, R5, R6]
print('the non-rephasing diagrams are R4, R5 and R6', nonrephasing)

In [ ]:
# constructing the Hamiltonian
hbar = 0.658211951  # in eV fs
Ee = 2  # eV -> put in energy of a few thiophenes
Ew = 0.2  # eV
J = 0.02 # eV hopping term its positive so we have a H aggregate
S = 1  # Huang-Rys factor
N = 3 # number of sites
n = 3 # number of phonon states for each site.

mu = 1.0  # dipole strength. Same for all sites

jz = jmat(1/2, 'z')
jm = jmat(1/2, '-')
jp = jmat(1/2, '+')

sz1 = tensor(jz,qeye(2), qeye(2), qeye(n), qeye(n), qeye(n))
sz2 = tensor(qeye(2),jz, qeye(2), qeye(n), qeye(n), qeye(n))
sz3 = tensor(qeye(2),qeye(2), jz, qeye(n), qeye(n), qeye(n))
a1 = tensor(qeye(2),qeye(2), qeye(2), destroy(n), qeye(n), qeye(n))
a2 = tensor(qeye(2),qeye(2), qeye(2), qeye(n), destroy(n), qeye(n))
a3 = tensor(qeye(2),qeye(2), qeye(2), qeye(n), qeye(n), destroy(n))
sp1 = tensor(jp,qeye(2), qeye(2), qeye(n), qeye(n), qeye(n))
sp2 = tensor(qeye(2), jp, qeye(2), qeye(n), qeye(n), qeye(n))
sp3 = tensor(qeye(2),qeye(2), jp, qeye(n), qeye(n), qeye(n))
sm1 = tensor(jm,qeye(2), qeye(2), qeye(n), qeye(n), qeye(n))
sm2 = tensor(qeye(2), jm, qeye(2), qeye(n), qeye(n), qeye(n))
sm3 = tensor(qeye(2),qeye(2), jm, qeye(n), qeye(n), qeye(n))

H = sz1*(Ee/2 + Ew*(a1.dag()*a1+1/2) + S*(a1.dag()+a1))+ \
sz2*(Ee/2 + Ew*(a2.dag()*a2+1/2) + S*(a2.dag()+a2))+ \
sz3*(Ee/2 + Ew*(a3.dag()*a3+1/2) + S*(a3.dag()+a3))+ \
J*(sp1*sm2 + sp1*sm3 + sp2*sm3 + sm1*sp2 + sm1*sp3 + sm2*sp3) # coupling terms

H = H/hbar


A = sm1+sm2+sm3
mu = A.dag()+A # dipole operator total

In [ ]:
print(mu)

In [ ]:
plt.figure()
plt.imshow(np.real(mu.full()))
plt.title('Total Dipole in initial basis')
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.real(H.full()))
plt.title('Total Hamiltonian in initial basis')
plt.colorbar()
plt.show()

In [ ]:
## SKIPPING THIS STEP FOR NOW

# creating collapse operators using system-bath couplings
kappa = 0.1  # internal relaxation
kB = 8.617333262*1e-5  # Boltzmann constant eV/K
T = 200  # temperature in K
kT = T*kB
beta = 1/kT
n1 = 1/(np.exp(E1*beta)-1)  # <n1>, thermal populations of the uncoupled states
n2 = 1/(np.exp(E2*beta)-1)  # <n2>, thermal populations of the uncoupled states
n3 = 1/(np.exp(E3*beta)-1)  # <n3>, thermal populations of the uncoupled states
c1 = np.sqrt(kappa*(n1+1))*a  # relaxation operators
c2 = np.sqrt(kappa*(n2+1))*b  # relaxation operators
c3 = np.sqrt(kappa*(n3+1))*c  # relaxation operators
c4 = np.sqrt(kappa*n1)*a.dag()  # excitation operators
c5 = np.sqrt(kappa*n2)*b.dag()  # excitation operators
c6 = np.sqrt(kappa*n3)*c.dag()  # excitation operators
c_ops = [c1, c2, c3, c4, c5,c6]  # collapse operator list

In [ ]:
# getting eigen states and energies
en,T = H.eigenstates()

In [ ]:
plt.figure()
plt.plot(en, 'o')
plt.xlabel('States')
plt.ylabel('Energies (eV)')
plt.title('Eigen energies')
plt.show()


In [ ]:
plt.figure()
plt.imshow(np.real(H.transform(T).full()))
plt.title('Total Hamiltonian in eigen basis')
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.real(mu.transform(T).full()))
plt.title('Total dipole in eigen basis')
plt.colorbar()
plt.show()

In [ ]:
# setting up initial state
rho = tensor(fock_dm(2, 0), fock_dm(2, 0), fock_dm(2,0), fock_dm(n,0), fock_dm(n,0), fock_dm(n,0))  # ground state of Hamiltonian
plt.figure()
plt.imshow(np.real(rho.full()))
plt.title('Initial state in initial basis')
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.real(rho.transform(T).full()))
plt.title('Initial state in initial basis')
plt.colorbar()
plt.show()

In [ ]:
sys = System(H=H.transform(T), rho=rho.transform(T), a=A, u=mu, c_ops=[], diagonalize=True)

In [ ]:
sys.diagram_donkey([0, 10, 20, 50], [R1])

In [ ]:
# generating 2Dcoherence response for rephasing
time_delays = [20, 10, 100]
scan_id = [0, 2]
response_list = []
total_diagrams = rephasing+nonrephasing
for k in range(6):
    states, t1, t2, dipole = sys.coherence2d(time_delays, total_diagrams[k], scan_id, r=1, parallel=True)
    response_list.append(1j*dipole)
spectra_list, extent, f1, f2 = sys.spectra(np.imag(response_list), resolution=1)
qsave(spectra_list, 'example1_res1_spectra_list')
qsave(response_list, 'exampel1_res1_response_list')
pf.multiplot(np.imag(spectra_list), extent, ['E emission', 'E absorption'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6'], 'log',
             color_map='PuOr')

# Note: 


1.   The silva_plots function has some issue with the labeling of the axis (i am looking into it now), that's why it was giving wrong energies for peaks (~ 1eV instead of ~2 eV).
2.   For now you can use local jupyter notebook/lab to run this file and using command %matplotlib to get the above plots in a new window. The advantage is that you get zoom function which can be use to explore all the diagrams (R1 through 6) easily.

3. Notice the peaks at very very low frequencies. In my understanding these are the consequences of the oscillator 1 with E1=0.01 eV. It would be interesting to see dipole allowed transitions on energy level diagram



In [ ]:
mud_mat = mu.transform(T).full()
print('states connected via dipole operator')
connected_states = []
for i in range(len(mud_mat)):
   connected_states += [(i,x) for x in np.where(np.abs(mud_mat[i])>1e-2)[0]]
# now these tuples are double counting the number of connections. 
# removing the double counting by requiring that second state always be greater than first one
connected_states = [x for x in connected_states if x[0]<x[1]]
print('total connected state pairs ', len(connected_states))
print('first 10 are ')
connected_states[:10]


### now i need to make a function for plotting lines on energy diagram from these tuples.

In [ ]:
plt.figure()
plt.plot(range(len(en)),en,'o')
plt.xlabel('States')
plt.ylabel('Energy (eV)')
plt.title('Eigenstates and energies along with dipole elements')

max_dipole = np.abs(np.max(mud_mat))
for k in connected_states:
  x,y = k
  ex = en[x]
  ey = en[y]
  lx = y-x
  ly = ey-ex
  plt.arrow(x, ex, y-x, ey-ex, lw=np.abs(mud_mat[x,y]), color=list([1,1-np.abs(mud_mat[x,y])/max_dipole,0]))
plt.show()

### Note: In the above figure, both color and thickness of lines represent the magnitude of the dipole operator element connecting the two states. This shows that the structure of dipole operator (and consequently its action) is a little complecated in this diagonalzied coupled harmonic system. Additionally, there energy eigenstates with almost similar energy are also coupled via dipole elements, which results in very low frequency peaks observed in R1 through R6.